In [ ]:
#LOAD DATA
from dataclasses import dataclass
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import matplotlib.pyplot as plt
from collections import Counter
from IPython.display import display
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (20, 7)

import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
#from numpy_ext import rolling_apply
from arch import arch_model

#from Quantapp.Algorithm   import Algorithm
#from Quantapp.Computation import Computation
#from Quantapp.Plot        import Plot
#from Quantapp.DataManager import DataManager
#from Quantapp.Universe    import Universe

time_frames = [10]

csv_file_paths = [
    'csv_files/S&P 500.csv',
]


period = '1y'
interval='1d'
# Retrieve benchmark data using yfinance
benchmark_ticker = 'SPY'
benchmark = yf.download(benchmark_ticker, period=period, interval=interval)
benchmark_returns = benchmark['Close'].pct_change().fillna(0)

# Retrieve risk-free rate data using yfinance
rf_ticker = '^IRX'
rf_data = yf.download(rf_ticker, period=period, interval=interval)
rf = rf_data['Close'].pct_change().fillna(0)  # Convert to returns

# If the data needs to be reindexed like benchmark
rf = rf.reindex_like(benchmark_returns)


In [ ]:
"""
market_sd = comp.rolling_windows(algorithm.standard_deviation,time_frames, benchmark['Close']) 
market_yz = comp.rolling_windows(algorithm.yang_zhang_volatility,time_frames, benchmark) 
metrics = pd.concat([market_sd, market_yz], axis=1).fillna(0)

scaler = StandardScaler()
scaler.fit(metrics)

standardized_metrics = pd.DataFrame(scaler.transform(metrics), columns=metrics.columns)

standardized_metrics
#px.addhline(y=0)
#market_sd.plot(title='Standard Deviation')
#market_yz.plot(title='Yang Zhang')
#standardized_metrics.plot()"""

In [ ]:
#am = arch_model(returns, p=1, o=1, q=1, power=1.0, dist="StudentsT")
arch_models = {}

arch_models['garch']      = arch_model(benchmark_returns)
arch_models['egarch']     = arch_model(benchmark_returns,vol='EGARCH')
arch_models['gjrgarch']   = arch_model(benchmark_returns, p=1, o=1, q=1)
arch_models['tgarch']     = arch_model(benchmark_returns, p=1, o=1, q=1, power=1.0)
arch_models['studenttgarch']   = arch_model(benchmark_returns, p=1, o=1, q=1, power=1.0, dist="StudentsT")

arch_model_results = {}

for key, value in arch_models.items():
    arch_model_results[key] = value.fit(disp="off")

arch_model_results["garch"].plot()
arch_model_results["gjrgarch"].plot()
#results = garch.fit(disp=off)
#results.summary()
#results.plot()